In [1]:
!pip install transformers datasets

In [2]:
from transformers import pipeline
task = "sentiment-analysis"

classifier = pipeline(task)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


In [3]:
from datasets import load_dataset
dataset = load_dataset("imdb")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
import numpy as np

# Define the range
start = 0
end = len(dataset["train"])
num_indices = 100

# Create a range of values and then randomly select from them
indices_range = np.arange(start, end)
random_indices = np.random.choice(indices_range, size=num_indices, replace=False)


In [5]:
classifier(dataset["train"][0]["text"]), dataset["train"][0]["label"]

([{'label': 'POSITIVE', 'score': 0.7872804403305054}], 0)

In [6]:
hits = 0
misses = 0
for index in random_indices:
  ix = int(index)
  sentiment = classifier(dataset["train"][ix]["text"][:512])
  text_label = 'POSITIVE' if dataset["train"][ix]["label"] == 1 else 'NEGATIVE'
  if sentiment[0]["label"] == text_label:
    hits += 1
  else:
    misses += 1
print(f"Accuracy: {hits / (hits + misses)}")

Accuracy: 0.79


In [7]:
!pip install flask requests

In [8]:
%%writefile app.py
from flask import Flask, request, jsonify
from transformers import pipeline

app = Flask(__name__)

task = "sentiment-analysis"
classifier = pipeline(task)

@app.route('/sentiment', methods=['POST'])
def predict():
    data = request.get_json()
    text = data['text']
    sentiment = classifier(text)[0]
    return jsonify(sentiment)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)

Writing app.py


In [9]:
!nohup python app.py &

nohup: appending output to 'nohup.out'


In [14]:
import requests
response = requests.post('http://localhost:5000/sentiment', json={'text': 'I love this movie!'})

In [15]:
response.json()

{'label': 'POSITIVE', 'score': 0.9998775720596313}